# Mounting Gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Libraries

In [ ]:
cd 'your code directory'

In [ ]:
import keras
import pickle
import numpy as np
from tqdm import tqdm 
from skimage import data
from scipy import ndimage
from skimage import filters
import matplotlib.pyplot as plt
from Functions import PlotingResults
import Functions 
from Functions import EvaluateMain 

# Load The Model

In [ ]:
### load the model 
BCDU_net_D3_model = keras.models.load_model('./my_en_de')
BCDU_net_D3_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 32  320         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 32  9248        ['conv2d[0][0]']                 
                                )                                                           

# Load Data 

In [ ]:
#### calculate the output of model 
def OutPutModel(my_data):
    my_out = []
    for i in tqdm(range(len(my_data))):
        my_out.append(BCDU_net_D3_model.predict(my_data[i][0,:,:,:]))
    return my_out


def Add_Dim(data):
    ################## add a dim to data
    data_scaled = []
    for j in tqdm(range(len(data))):
            output = np.zeros((1,data[j].shape[0],data[j].shape[1],data[j].shape[2],1))
            output[0,:,:,:,0] = data[j][:,:,:] 
            data_scaled.append(output)
    return data_scaled


def LoadData(DatasetName):

    ############## Load 
    if(DatasetName=='Mosmed_Dataset'): 
        Base_data = np.load('/PreprocessedData/BaseCTImages.npy')
            
        data = np.load('/PreprocessedData/LungImages.npy')

        mask = np.load('/PreprocessedData/InfectionMasks.npy')

        LungMask = np.load('/PreprocessedData/LungMask.npy')

        for i in range(mask.shape[0]):
            mask[i,:,:,:]=np.where(mask[i,:,:,:]>0.5,1,0) 
            LungMask[i,:,:,:]=np.where(LungMask[i,:,:,:]>0.0005,1,0)


    if(DatasetName=='UESTC'):
        Base_data = np.load('./BaseCTImages.npy')
            
        data = np.load('./LungImages.npy')

        mask = np.load('./InfectionMasks.npy')

        LungMask = np.load('./LungMask.npy')
        for i in range(len(mask)):
            mask[i]=np.where(np.array(mask[i])>0.5,1,0) 
            LungMask[i]=np.where(np.array(LungMask[i])>0.0005,1,0)


    if(DatasetName=='Radiopedia'):
        Base_data = np.load('./BaseCTImages.npy')
            
        data = np.load('./LungImages.npy')

        mask = np.load('./InfectionMasks.npy')

        LungMask = np.load('./LungMask.npy')
        for i in range(len(mask[0].shape[0])):
            mask[i]=np.where(np.array(mask[i])>0.5,1,0) 
            LungMask[i]=np.where(np.array(LungMask[i])>0.0005,1,0)



    ####################### Add a dim
    ########### add a dim to data
    Data_scaled = Add_Dim(data)
    
    ########### add a dim to mask
    mask_scaled = Add_Dim(mask)
    
    ### get output of model 
    my_out = OutPutModel(Data_scaled)

    return Data_scaled,my_out, mask_scaled, LungMask





# Evaluation

In [ ]:
#### upload data  
my_data, my_out, mask_scaled, LungMask = LoadData("Mosmed_Dataset")

####### Calculating the evaluation metrics(Dice score, Specificity, Sensitivity )

dice_list, sensitivity_list, specificity_list, InfectionPercent = EvaluateMain(my_data,my_out,mask_scaled,LungMask,Threshold= 0.2,thresholdMethod='1',
                                                                                      PlotingCase=10,PlotingSlice=15,PostPro=False)


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: Mean of empty slice.
100%|██████████| 2/2 [00:00<00:00, 69.56it/s]


# Plot The Results

In [ ]:
############### ploting resuts

### Dice score  
PlotingResults(dice_list ,InfectionPercent,ScoreName='Dice score')

##### Sensitivity
PlotingResults(sensitivity_list,InfectionPercent,ScoreName='Sensitivity')


##### Specificity
PlotingResults(specificity_list,InfectionPercent,ScoreName='specificity')



Dice score Mean: 63.2789490773616 Dice score Std: 13.67149748679442
Sensitivity Mean: 69.93464406636804 Sensitivity Std: 11.529107714983061
specificity Mean: 97.82896349827183 specificity Std: 1.7237497375248287


In [ ]:
!pip list

Package                       Version
----------------------------- --------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   21.1.0
arviz                         0.11.4
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.1.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4                           0.0.1
CacheControl